In [9]:
using LinearAlgebra, Plots, DataFrames, Distributions
import CSV
upscale = 1
default(line = (:steppre, 0.5, 1, 2), size=(800*upscale,600*upscale))
plotly()
include("../KalmanFilter.jl")
import .KalmanFilter


### Messdaten
Im nächsten Schritt generieren wir messdaten.

In [34]:
series = CSV.File("SensorTracker_20181213_154327.csv", delim = ";") |> DataFrame
deletecols!(series, :Proximity)
deletecols!(series, :Light)
deletecols!(series, :Pressure)
first(series, 2)

,Timestamp,Accel_x,Accel_y,Accel_z,Gyro_x,Gyro_y,Gyro_z,Magnetic_x,Magnetic_y,Magnetic_z,Latitude,Longitude,Elevation,Speed
,String⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰
1,20181213_154327,0.227,8.327,4.765,0.118,0.08,0.11,57.8,-34.8,-110.5,48.9322,12.6222,376.9,65.1
2,20181213_154327,0.34,9.591,4.7,0.04,-0.038,-0.028,57.6,-34.8,-110.6,48.9322,12.6222,376.9,65.1


In [4]:
describe(series)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Int64,DataType
1,Timestamp,,20181213_154327,,20181213_163541,3135,0,String
2,Accel_x,0.276942,-5.052,0.225,4.942,,0,Float64
3,Accel_y,8.95713,4.197,8.959,15.125,,0,Float64
4,Accel_z,4.07739,-4.319,4.085,14.208,,0,Float64
5,Gyro_x,0.000470847,-0.674,-0.0,1.019,,0,Float64
6,Gyro_y,-0.001933,-19.367,0.001,0.456,,0,Float64
7,Gyro_z,-0.0067118,-11.429,-0.005,0.321,,0,Float64
8,Magnetic_x,56.8806,3.7,57.2,100.1,,0,Float64
9,Magnetic_y,-1.02245,-104.1,20.1,49.0,,0,Float64


### Zustandsvektor

$$
    x_k = 
    \begin{pmatrix}
    a_x \\ a_y \\ a_z \\ v_x \\ v_y \\ v_z \\ s_x \\ s_y \\ s_z
    \end{pmatrix}
$$
$$
    A =
    \begin{pmatrix}
    1 & 0 & 0
    \end{pmatrix}
$$

### Bestimmung der Sensor Rauschens

Versuch: Das Handy liegt ruhig auf dem Tisch. Wir beobachten die Änderung  der Sensordaten über 60 Sekunden.

In [41]:
calibration = CSV.File("sensor_dump_calibration.csv", delim = ";") |> DataFrame
deletecols!(calibration, :Latitude)
deletecols!(calibration, :Longitude)
deletecols!(calibration, :Elevation)
deletecols!(calibration, :Speed)
deletecols!(calibration, :Pressure)
describe(calibration)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Union…,Int64,DataType
1,Timestamp,,20190311_212841,,20190311_212940,60,0,String
2,Accel_x,-0.190566,-0.22,-0.192,-0.141,,0,Float64
3,Accel_y,-0.0310093,-0.069,-0.031,0.005,,0,Float64
4,Accel_z,9.75633,9.711,9.757,9.773,,0,Float64
5,Gyro_x,8.77193e-5,-0.004,0.0,0.005,,0,Float64
6,Gyro_y,5.36636e-5,-0.02,-0.0,0.004,,0,Float64
7,Gyro_z,-4.23117e-5,-0.004,-0.0,0.003,,0,Float64
8,Magnetic_x,-4.53168,-5.2,-4.7,-3.0,,0,Float64
9,Magnetic_y,-45.4118,-46.0,-45.5,-44.3,,0,Float64


$$
z = 
$$

In [46]:
plot(calibration[:Accel_x], yticks = 0:10)
plot!(calibration[:Accel_y])
plot!(calibration[:Accel_z])

### Glättung der X-Achse

In [36]:
# Populationsmittelwert wird abgelesen
median_x = 0.276942
# Damit berechnen wir die empirische Varianz
var_accel_x = 1/length(series[:Accel_x]) * reduce(+, map(x -> (x - median_x)^2, series[:Accel_x]))

0.8023490574157556

In [24]:
y_x = series[8200:8500, :Accel_x]
y_y = series[8200:8500, :Accel_y]
y_z = series[8200:8500, :Accel_z]

y = [y_x y_y y_z]
x = collect(1:size(y)[1])

A = [.7 0 0; 0 .7 0; 0 0 .7] # Zustandsübergangsmatrix
Q = [.1 0 0; 0 .2 0; 0 0 .2]

G = [.8 0 0; 0 .8 0; 0 0 .8] # Messabbildung Matrix
R = [.5 0 0; 0 .5 0; 0 0 .5]

x̂_0 = [0; 0; 0]
# x̂_0 = f(1)
Σ_0 = [.1 0 0; 0 .1 0; 0 0 .1]

model = KalmanFilter.Kalman(A, Q, G, R, x̂_0, Σ_0)


for (index, value) in enumerate(x)
    y_cur = y[index, :]
    
    fusioned = KalmanFilter.fusion(model, y_cur)
    y_filtered[:, index] = fusioned[1]
    
    predicted = KalmanFilter.predict(model)
    y_predicted[:, index] = predicted[1]
end
error = y_filtered' - y
@show sum(error, dims = 1)
@show sum(error) / size(y)[1]

sum(error, dims=1) = [-294.871 -796.508 -337.164]
sum(error) / (size(y))[1] = -4.745989762347106


-4.745989762347106

301-element Array{Float64,1}:
 -0.06652482269503546  
  0.0029269929533579445
 -0.14126492902294113  
 -0.1587617562106944   
 -0.15050221489181245  
 -0.23934800610142226  
 -0.07196734896097563  
 -0.16931432010370875  
 -0.07373607299200513  
 -0.029248024172245264 
 -0.021052413701226395 
  0.025848123926142483 
  0.1529115013926106   
  ⋮                    
  0.5747965493157398   
  0.7837438715989021   
  0.6435973922465128   
  0.5782830870891167   
  0.3469782207551399   
  0.386572484766747    
  0.3634610497404369   
  0.42367504599487327  
  0.30711505284479246  
  0.28388071646230606  
  0.24399111142192806  
  0.14812961590277726  

In [32]:
plot()
plot!(x, y_filtered[1, :]', label = "Filtered")
plot!(x, y, label = "Ground Thruth", linestyle = :dot)

BoundsError: BoundsError: attempt to access 1-element Array{Float64,1} at index [1:601]

In [15]:
y_x_gyro = series[8200:9000, :Gyro_x]
y_y_gyro = series[8200:9000, :Gyro_y]
y_z_gyro = series[8200:9000, :Gyro_z]

y_gyro = [y_x_gyro y_y_gyro y_z_gyro]
x = collect(1:size(y_gyro)[1])

A = [1 0 0; 0 1 0; 0 0 1] # Zustandsübergangsmatrix
Q = [.3 0 0; 0 .3 0; 0 0 .3]

G = [1 0 0; 0 1 0; 0 0 1] # Messabbildung Matrix
R = [1.8 0 0; 0 1.8 0; 0 0 1.8]

x̂_0 = [1; 1; 1]
Σ_0 = [.1 0 0; 0 .1 0; 0 0 .1]

model = KalmanFilter.Kalman(A, Q, G, R, x̂_0, Σ_0)

y_filtered_gyro = zeros(3, length(x))
y_predicted_gyro = zeros(3, length(x))
for (index, value) in enumerate(x)
    y_cur = y_gyro[index, :]

    fusioned = KalmanFilter.fusion(model, y_cur)
    y_filtered_gyro[:, index] = fusioned[1]
    
    predicted = KalmanFilter.predict(model)
    y_predicted_gyro[:, index] = predicted[1]
end
error = y_filtered' - y
@show sum(error, dims = 1)
@show sum(error) / size(y)[1]

sum(error, dims=1) = [-82.0212 -204.194 -86.4702]
sum(error) / (size(y))[1] = -1.238155920773013


-1.238155920773013

In [16]:
plot(x, y_predicted_gyro', label = "Predicted", linestyle=:dash)
plot!(x, y_filtered_gyro', label = "Filtered")
plot!(x, y_gyro, label = "Ground Thruth", linestyle = :dot)